In [1]:
import pandas as pd
import torch
import numpy as np
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [64]:
df = pd.read_csv('observations.csv.gz')
df.head()

,EpochID,Svid,UTCTime,Cn0DbHz,PseudorangeRateMetersPerSecond,Residual,SatDistance,PseudoRange,elevation,Label
0,1,E02,2020-02-11T08:49:27Z,23.471855,441.508879,NaN,2.737755e+07,NaN,16.975217,NLOS
1,1,E07,2020-02-11T08:49:27Z,45.905182,297.262449,NaN,2.434706e+07,NaN,52.882825,LOS
2,1,E08,2020-02-11T08:49:27Z,24.197746,491.962844,NaN,2.866219e+07,NaN,4.721491,NLOS
3,1,E11,2020-02-11T08:49:27Z,NaN,NaN,NaN,2.868801e+07,NaN,4.387583,NLOS
4,1,E12,2020-02-11T08:49:27Z,22.291088,47.024307,NaN,2.669391e+07,NaN,23.791947,NLOS


In [65]:
df  = df.loc[:,['Cn0DbHz','Residual', 'Label']]
df = df.dropna()

In [66]:
def transf_labels(l1):
    if l1 == 'NLOS':
        return 0
    else:
        return 1
df['Label'] = df['Label'].apply(transf_labels)

In [101]:
df1 = df.loc[:,['Cn0DbHz','Residual','Label']]
trainy = df1['Label']
trainx =  df1.drop(['Label'], axis=1)

In [102]:
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)
train_x = torch.from_numpy(trainx[:100000].values)
train_y = torch.from_numpy(trainy[:100000].values)
test_x = torch.from_numpy(trainx[100000:].values)
test_y = torch.from_numpy(trainy[100000:].values)

In [103]:
test_y

tensor([0, 1, 1,  ..., 0, 0, 1])

In [207]:
import torch.utils.data as Data

class GNSSDataset(Data.Dataset):
    def __init__(self, X, y):
        self.data = X
        self.target = y.long()
    
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        return x ,y
        
    def __len__(self):
        return len(self.data)
        

In [214]:
class BaseNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(nn.Linear(2,50), nn.ReLU())
        self.linear2 = nn.Sequential(nn.Dropout(0.4), nn.Linear(50,50), nn.ReLU(), nn.Linear(50,50))
        self.output = nn.Linear(50, 2)
        self.sgmd = nn.Sigmoid()
        
    def forward(self, x):
        x  = self.linear(x.float())
        x = self.linear2(x.float())
#         x = self.linear(x.float())
#         output = torch.argmax(self.sgmd(self.output(x)), 1)
        output = self.sgmd(self.output(x))
        return output

In [223]:
nn1 = BaseNN()
lossf = nn.CrossEntropyLoss()
epochs = 100

In [224]:
trainDataset = GNSSDataset(train_x, train_y)
trainLoader = Data.DataLoader(dataset=trainDataset, batch_size=500, shuffle=True)

In [225]:
from sklearn.metrics import roc_auc_score
from torch.autograd import Variable

def TrainNN(dataloader, model, num_epochs, loss_f, lr):
    for i in range(num_epochs):
        for step, (items, labels) in enumerate(dataloader):
            input_x = Variable(items)
            labels_y = Variable(labels)
            output = model(input_x)
            loss = loss_f(output, labels_y)
            optimiser = torch.optim.Adam(model.parameters(), lr)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            
            if step % 750 == 0:
                test_output = model(test_x)
                pred_y = torch.max(test_output , 1)[1].data.squeeze()
                print('Test roc auc:' + str(roc_auc_score(test_y, pred_y)))
                accuracy = (pred_y == test_y).sum().item() / float(test_y.size(0))
                print(accuracy)

In [227]:
lrs = [1e-3]
for v1 in lrs:
    TrainNN(trainLoader, nn1 , epochs, lossf, v1)

Test roc auc:0.7000012270011404
0.6605487228003785
Test roc auc:0.7391015780705604
0.7071901608325449
Test roc auc:0.7413656327176648
0.7101702932828761
Test roc auc:0.74536977689346
0.715752128666036
Test roc auc:0.7493016257600658
0.7196310312204351
Test roc auc:0.7475202311520313
0.7170293282876065
Test roc auc:0.7500602285798057
0.7202932828760643
Test roc auc:0.7509442815310354
0.7217123935666983
Test roc auc:0.7535244328680586
0.7244087038789025
Test roc auc:0.7555265049559563
0.7271996215704825
Test roc auc:0.7538435034140553
0.7255912961210974
Test roc auc:0.7596254927567601
0.7320245979186376
Test roc auc:0.7580653512136362
0.7298959318826869
Test roc auc:0.7572792116743339
0.7292336802270577
Test roc auc:0.7625376196219902
0.7354777672658467
Test roc auc:0.7622056440937834
0.734720908230842
Test roc auc:0.7597904252816102
0.7318826868495742
Test roc auc:0.7660986820829172
0.7398297067171239
Test roc auc:0.7636405273671001
0.7361873226111637
Test roc auc:0.7611143942299481
0.7